In [ ]:
import $ivy.`org.typelevel::cats-core:2.1.0`

// These are all the imports you need for everything here
import cats.implicits._
import cats.data.Writer

# The Writer Monad
`cats.data.Writer` is a monad that lets us carry a log along with the computation and extract the log along with the final result. A common use for `Writer` is recording sequences of steps in a multi-threaded computation where standard imperative logging techniques can result in interleaved messages from different contexts.

A `Writer[W, A]` carries two values, the log of type `W` and a result of type `A`.

In [ ]:
// We can create a writer with logs and a result using its constructor
val writer1 = Writer(Vector(
  "It was the best of times",
  "it was the worst of times"
), 1859)

// Or we can use this syntax
val writer2 = 1859.writer(Vector("It was the best of times", "it was the worst of times"))

// Or we can create it using the pure syntax, as long as we have Monoid[W] in scope to create an empty log
type Logged[A] = Writer[Vector[String], A]
val writerNoLog = 123.pure[Logged]

// We can create a writer with a log but no result
val writerNoResult = Vector("msg1", "msg2", "msg3").tell

// We can extract the result and log using .value and .written respectively
val log1 = writer1.written
val result1 = writer1.value

// Or we can get both at once using .run
val (log2, result2) = writer1.run

// We can use map and flatMap since it's a monad
val writer3 = for {
  a <- 10.pure[Logged]
  _ <- Vector("a", "b", "c").tell
  b <- 32.writer(Vector("x", "y", "z"))
} yield a + b

val result3 = writer3.run

// We can use mapWritten to map on the logs
val mapWritten = writer1.mapWritten(_.map(_.toUpperCase)).run

// We can transform both the logs and result at once using bimap or mapBoth
val bimap = writer1.bimap(
  log => log.map(_.toUpperCase),
  res => res * 100
).run

val mapBoth = writer1.mapBoth { (log, res) =>
  val log2 = log.map(_.toUpperCase)
  val res2 = res * 100
  (log2, res2)
}.run

// We can clear the log with reset
val reset = writer1.reset.run

// And we can swap the log and result with swap
val swap = writer1.swap.run